# 1. Enkratni

Napiši funkcijo `enkratni(s)`, ki dobi neprazen seznam seznamov števil. Števila znotraj posamičnega seznama se ne ponavljajo. Funkcija vrne seznam, ki vsebuje največ števil, ki jih ne vsebuje noben drug seznam. Če je takšnih seznamov več, vrne prvega med njimi.

Klic `enkratni([[1, 2, 3], [4, 1, 5], [2, 8], [3, 1, 2, 8]])` vrne `[4, 1, 5]`, saj ta seznam vsebuje dve števili (`4` in `5`), ki jih ne vsebuje noben drug seznam.

## Rešitev

Tole je naloga iz slovarjev ali iz množic. Ali brez; v tem primeru je to naloga iz zoprnega gnezdenja zank in pogojev.

### Štetje s slovarji

Nalogo lahko rešimo tako, da za vsako število preštejemo, kolikokrat se pojavi. Nato gremo čez vse sezname in za vsak seznam preštejemo, koliko število v njem se je - prek vseh seznamov - pojavilo le enkrat, torej le v njem. Vrnemo seznam z največ takšnimi števili.

In [1]:
from collections import defaultdict

def enkratni(ss):
    # Preštejemo, kolikokrat se pojavi katero število
    pojavitev = defaultdict(int)
    for s in ss:
        for x in s:
            pojavitev[x] += 1

    naj_s = None
    naj_uni = 0
    for s in ss:
        # Za vsak seznam preštejemo, koliko njegovih
        # elementov se pojavi le enkrat (torej v njem)
        uni = 0
        for x in s:
            if pojavitev[x] == 1:
                uni += 1
                
        # In si zapomnimo seznam z največ takšnimi števili
        if uni > naj_uni:
            naj_uni = uni
            naj_s = s
    return naj_s

Za izpit je tole kar dolga funkcija. Spretnejši znajo hitreje prešteti, koliko elementov nekega seznama se pojavi le enkrat: `sum(pojavitev[x] == 1 for x in s)`. Še spretnejši poznajo lambde. Najspretnejši poznajo `chain` in vedo, kaj pomenijo zvezdice v seznamih argumentov. Zato nalogo rešijo takole:

In [2]:
from collections import Counter
from itertools import chain

def enkratni(ss):
    pojavitev = Counter(chain(*ss))
    return max(ss, key=lambda s: sum(pojavitev[x] == 1 for x in s))

### Razlike množic

Očitnejša rešitev je, da jemljemo posamične sezname in iz njih brišemo vse elemenete, ki se pojavijo tudi v kakem drugem seznamu. Nekaj podobnega je naredila Zoja; skoraj ji je uspelo.

In [3]:
def enkratni(ss):
    najboljsa = ss[0]
    najvec = 0
    for s in ss:
        edinstveno = set(s)
        for t in ss:
            if t is not s:
                edinstveno -= set(t)
                
        if len(edinstveno) > najvec:
            najvec = len(edinstveno)
            najboljsa = s
            
    return najboljsa

Drugi del je očiten; zanimivejši je prvi. Elemente `s` smo shranili prepisali v množico, da jih bomo lažje odstranjevali. Nato odštejemo od te množice elemente vseh množic, *ki niso `t`*.

Pazite, ne pišemo `if t != s`, saj se lahko zgodi, da imamo dva enaka seznama in v tem primeru moramo odšteti (vse!) elemente. Ne `t`in `s` morata biti neista seznama.

Zoja je pisala nekaj takšnega:

```
    for i in range(len(ss)):
        for j in range(len(ss)):
            if ss[i] != ss[j]:
```

Tudi to je narobe, saj preverja enakost. (Njen program je imel sicer še drug, malo hujši problem.) Pač pa bi bilo pravilno pisati 

```
    for i in range(len(ss)):
        for j in range(len(ss)):
            if i != j:
```

Nekdo je poskušal nekaj podobnega tako, da je iskal pare seznamov, ki vsebujejo kak isti element in ga v tem primeru odstranil iz obeh seznamov, torej nekaj v slogu


```
    for s in ss:
        for x in s:
            for t in ss:
                if x in t:
                    s.remove(x)
                    t.remove(x)
```

Predtem se je (pohvalno!) spomnila narediti kopijo seznamov. Rešitev ima dva problema. Prvi je, da se `x` lahko pojavi v več seznamih. V tem primeru bomo `s.remove(x)` poklicali večkrat in v drugem klicu se bo Python pritožil, da `s` ne vsebuje `x`. V izogib temu je študentka dodala še `break`, kar pa spet ni dobro, saj v primeru, da se `x` pojavi v treh seznamih, pobriše le njegovi pojavitvi v prvih dveh.

Druga težava je, da znotraj zanke `for x in s` kličemo `s.remove(x)`. To ne deluje: ko pobrišemo element seznama, se ostali elementi pojavijo za eno mesto nižje, zanka pa nadaljuje z elementom na mestu, ki je sledil elementu `x` - in pri tem preskoči `x`-ovega naslednika, ki se je izmuznil na mesto, kjer je bil prej `x`.

### Telovadba z zankami

Če nočemo vedeti ne za slovarje ne za množice, se bomo pretepali z zankami in pogoji. Takole.

In [4]:
def enkratni(ss):
    naj_s = None
    naj_uni = 0
    for s in ss:  # Za vsak seznam
        uni = 0
        for x in s:  # Za vsako število iz tega seznama
            for t in ss:  # gremo čez vse sezname,
                # da poiščemo takšnega, ki vsebuje to število
                if x in t and t is not s:
                    break
            else:
                uni += 1
                
        if uni > naj_uni:
            naj_uni = uni
            naj_s = s
            
    return naj_s

Ključen je pogoj `if x in t and t is not s`, ki se sprašuje, ali nek `t` (ki ni `s`) vsebuje `x`. Če je tako, z `break` prekinemo zanko. Po zanki je `else`, ki se izvede, če zanka ni bila prekinjena z `break` - torej, če je element unikaten.

Telovadbi z `break`-i in `else`-i se da pogosto izogniti s funkcijo `all` ali `any`. V tem primeru takole:

In [5]:
def enkratni(ss):
    naj_s = None
    naj_uni = 0
    for s in ss:  # Za vsak seznam
        uni = 0
        for x in s:
            if not any(x in t for t in ss if t is not s):
                uni += 1
        if uni > naj_uni:
            naj_uni = uni
            naj_s = s
    return naj_s

Potem nas loči le še korak od

In [6]:
def enkratni(ss):
    naj_s = None
    naj_uni = 0
    for s in ss:
        uni = sum(not any(x in t for t in ss if t is not s) for x in s)
        if uni > naj_uni:
            naj_uni = uni
            naj_s = s
    return naj_s

Tu pa se ustavimo.

# 2. Vsota

Napiši funkcijo `vsota(s)`, ki prejme seznam, katerega elementi so cela števila in seznami števil in seznamov. Primer takšnega argumenta je `[[1, 2], [[3, 4], 5, 113, [12]], [[[4]]], [-2, 1]]`. Funkcija vrne vsoto vseh števil; v tem primeru `143`.

Če želimo preveriti, ali je `x` `int` (ali `list`), uporabimo `if isinstance(x, int)` (ali `isinstance(x, list)`).

## Rešitev

Rešitev klasične naloge iz rekurzije je

In [7]:
def vsota(s):
    v = 0
    for x in s:
        if isinstance(x, int):
            v += x
        else:
            v += vsota(x)
    return v

Ali, krajše,

In [8]:
def vsota(ss):
    return sum(x if isinstance(x, int) else vsota(x) for x in ss)

Rešitev, ki so se rekurziji izognile tako, da so ročno nagnezdile po pet zank, nisem upošteval kot pravilne.

# 3. Imena

Napiši funkcijo `imena(zelena, prepovedana)`, ki prejme seznam želenih imen in množico prepovedanih imen.
Če nobeno od želenih imen ni prepovedano, funkcija vrne kar podani seznam zelena. Sicer pa vrne seznam imen, v katerem je vsakemu imenu dodan presledek in številka v oklepaju. Vsa imena imajo pripeto isto številko. Številka mora biti najnižje pozitivno celo število, pri katerem nobeno od tako dopolnjenih imen ni prepovedano.

Če pokličemo `imena(["ana", "berta", "cilka"], {"berta", "ana (4)", "berta (2)", "cilka (2)", "ana (1)", "dani (7)"})`, funkcija vrne seznam `["ana (3)", "berta (3)", "cilka (3)"]`. Številke ne more uporabiti, ker je prepovedano `ana (1)`, številke 2 pa, ker sta prepovedani `berta (2)` in `cilka (2)`.

## Rešitev

Nalogo z najmanj kompliciranja rešimo tako, da lepo po vrsti poskušamo možna števila.

In [9]:
def imena(zelena, prepovedana):
    predlog = zelena
    i = 0
    while set(predlog) & prepovedana:
        i += 1
        predlog = [f"{x} ({i})" for x in zelena]
    return predlog

Mimogrede povadimo množice in oblikovanje nizov. Šlo bi tudi brez, a z njimi je očitno lažje.

Ne spreglejte, kako elegantno smo opravili s prvim pogojem, ki pravi, da v primeru, da nobeno ime ni prepovedano, vrnemo prvotni seznam. Funkcija sestavlja predloge; prvi predlog je začetni seznam in zanka teče, dokler ne sestavi sprejemljivega predloga.

# 4. Intervali

Napiši funkcijo `intervali(s)`, ki prejme neprazen niz z opisom nekih intervalov, na primer `"1-3, 5-9, 10-19, 21-21"`. Intervali so ločeni z vejico in presledkom ter sestavljeni iz dveh celih števil, ločenih z minusom, pri čemer je gornja meja večja ali enaka spodnji.

Funkcija mora vrniti niz, v katerem vsakemu intervalu sledita presledek in oklepaj, v katerem je zapisana širina intervala. Na koncu doda še `" => "` in nato vsoto dolžin vseh intervalov.

Klic `intervali("1-3, 5-9, 10-19, 21-21")` vrne `"1-3 (2), 5-9 (4), 10-19 (9), 21-21 (0) => 15"`.

## Rešitev

Naloga iz dela z nizi.

In [10]:
def intervali(opis):
    novi = []
    skupaj = 0
    for x in opis.split(", "):
        od, do = x.split("-")
        dolzina = int(do) - int(od)
        skupaj += dolzina
        novi.append(f"{od}-{do} ({dolzina})")
    return ", ".join(novi) + f" => {skupaj}"

# 5. Unikado

Unikado je podoben pikadu. Tekmovalci eden za drugim mečejo puščice v tarčo. Ko vsak vrže po eno puščico, mečejo naslednjo. Število krogov je nepomembno.

Tarča je razdeljena na polja. Vsako polje na tarči prinaša določeno število točk; na tarči ni dveh polj z enakim številom točk. Tekmovalec ima toliko točk, kolikor je **vsota polj, v katera so zapičene njegove puščice**. Posebnost unikada pa je: **če tekmovalec zadane polje, v katerem je že od prej kakšna druga puščica, se ta, prejšnja puščica odstrani**. 

Napiši razred `Unikado` z naslednjimi metodami:

- Konstruktor prejme število tekmovalcev.
- Metodo `met(self, tocke)` pokličemo ob vsakem metu; argument pove število točk v polju, ki ga je zadel tekmovalec. Če je tekmovalec zgrešil tarčo (se zgodi), je tocke enak None.
- Metoda `stanje(self)` vrne seznam, ki vsebuje toliko elementov, kolikor je tekmovalcev, pri čemer elementi predstavljajo število točk, ki jih ima tekmovalec.
- Metoda `zmagovalec(self)` vrne zaporedno številko tekmovalca, ki ima največ točk. Tekmovalce štejemo od 1 naprej. Če si eden ali več tekmovalcev deli isto število točk, vrne None.

## Rešitev

Odebeljeni tisk je bil namig o tem, kaj mora hraniti razred in kaj se mora dogajati ob metih: razred mora vedeti, katera puščica tiči v polju s posamezno številko; ker številk ne poznamo vnaprej in ker ne gre nujno za zaporedna števila, bo za to najprikladnejše uporabiti slovar. Poleg tega bo moral razred, kot v zadnji domači nalogi, vedeti, koliko igralcev imamo in kdo je na vrsti.

Ko se odločimo glede atributov, so metode preproste:
- `met` določi novega "lastnika" polja (razen, če je takmovalec vrgel `None`) in poveča zaporedno številko tekmovalca.
- `stanje` pripravi seznam s točkami, nato pa gre prek polj in prišteva točke (ključe slovarja) tekmovalcem (vrednostim).
- `zmagovalec` pa na kakršenkoli že način poišče indeks elementa z največjo vrednostjo.

In [11]:
class Unikado:
    def __init__(self, tekmovalcev):
        self.tekmovalcev = tekmovalcev
        self.tekmovalec = 0
        self.lastniki = {}
        
    def met(self, tocke):
        if tocke is not None:
            self.lastniki[tocke] = self.tekmovalec
        self.tekmovalec = (self.tekmovalec + 1) % self.tekmovalcev

    def stanje(self):
        tockovalnik = [0] * self.tekmovalcev
        for tocke, lastnik in self.lastniki.items():
            tockovalnik[lastnik] += tocke
        return tockovalnik

    def zmagovalec(self):
        s = self.stanje()
        naj = max(s)
        if s.count(naj) > 1:
            return None
        return s.index(max(s)) + 1